Cyberdyne runs a production line with 8 machines M1 to M8. The products that go on this
line require processing by all 8 machines in sequence (i.e. starting on M1 then processing on
M2 and onwards through the rest of the machines until they finish processing on M8) but
the time that each product requires on each machines varies. The company has received an
order for 30 jobs (see table 1 - an Excel copy is also available on Blackboard).

### 0b. import liblaries

In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pulp import *

### 0a. load data

In [86]:
jobs = pd.read_csv('Jobs.csv')

In [87]:
jobs.set_index('Job')

,Time on M_1,Time on M_2,Time on M_3,Time on M_4,Time on M_5,Time on M_6,Time on M_7,Time on M_8
Job,,,,,,,,
1,9,3,6,10,8,5,3,2
2,2,4,4,1,1,5,5,3
3,5,9,8,2,9,8,1,10
4,3,5,10,6,8,9,1,2
5,2,2,7,9,7,8,8,1
6,1,5,7,6,6,5,2,10
7,8,1,2,1,6,6,4,2
8,9,3,2,4,6,3,6,10
9,4,4,2,3,7,7,10,8


### 1. Create a computer tool to calculate the time of the end of the final job on the las machine, given a sorted list of jobs. Assume that jobs cannot overtake one another in the line.

In [88]:
sorted_list_of_jobs = range(0,30)

In [109]:
def total_time(list_of_jobs):
     
    time_passed = 0
    n = 8 # number of machines
    
    all_jobs_finished = False
    job_stage = np.zeros(len(list_of_jobs), dtype=int)
    job_time_left = np.zeros(len(list_of_jobs), dtype=int)
    is_job_busy = np.zeros(len(list_of_jobs), dtype=bool)
    
    
    machine_time_left = np.zeros(n, dtype=int)  # specifies how much left for the job
    machine_job_number = np.zeros(n, dtype=int)  # specifies which job INDEX from list_of_jobs is being executed
    is_machine_busy = np.zeros(8, dtype=bool)
    
    while not all_jobs_finished:
        
        
        # iterate through machines to fill them with jobs
        for ind in range(0,n):
             
            # add appropraite amount of time to machines_time_left
            if not is_machine_busy[ind] and machine_job_number[ind] != -1:
                
                # get ind of next job
                next_job = machine_job_number[ind]
                # check wether next_job is busy
                if not is_job_busy[next_job] and job_stage[next_job] == ind:
                    
                    job_stage[next_job] += 1
                    is_job_busy[next_job] = 1
                    machine_job_number[ind] += 1
                    
                    job_time_left[next_job] += jobs.iloc[list_of_jobs[next_job],ind+1] 
                    machine_time_left[ind] += jobs.iloc[list_of_jobs[next_job],ind+1] 
                    print("Machine: " + str(ind) + " - " + str(machine_job_number[ind]))
                    
                    # if machine took its last item then tell it not to take any more
                    if machine_job_number[ind] >= len(list_of_jobs):
                         machine_job_number[ind] = -1
             
        print("--------------")  # this indicates that one unit time passed
        # decrement all machines in machines_time_left by 1
        time_passed += 1
        machine_time_left = np.where(machine_time_left < 1, machine_time_left, machine_time_left - 1)
        job_time_left = np.where(job_time_left < 1, job_time_left, job_time_left - 1)
        
        # update which machines are ready to take new jobs 
        is_machine_busy = [machine != 0 for machine in machine_time_left]
        is_job_busy = [job != 0 for job in job_time_left]
                 
                 
        # all jobs finished
        all_jobs_finished = all(job == 8 and busy == False for job in job_stage for busy in is_job_busy)
        
        # stop if your program enters infinite loop
        if (time_passed > 400):
            all_jobs_finished = True
    
    return time_passed


In [110]:
total_time(sorted_list_of_jobs)

Machine: 0 - 1
--------------
--------------
--------------
--------------
--------------
--------------
--------------
--------------
--------------
Machine: 0 - 2
Machine: 1 - 1
--------------
--------------
Machine: 0 - 3
--------------
Machine: 1 - 2
Machine: 2 - 1
--------------
--------------
--------------
--------------
Machine: 0 - 4
Machine: 1 - 3
--------------
--------------
Machine: 2 - 2
Machine: 3 - 1
--------------
Machine: 0 - 5
--------------
--------------
Machine: 0 - 6
--------------
Machine: 0 - 7
--------------
--------------
--------------
Machine: 1 - 4
Machine: 2 - 3
--------------
--------------
--------------
Machine: 3 - 2
Machine: 4 - 1
--------------
--------------
Machine: 0 - 8
Machine: 1 - 5
--------------
--------------
Machine: 1 - 6
--------------
Machine: 2 - 4
Machine: 3 - 3
--------------
--------------
--------------
Machine: 4 - 2
Machine: 5 - 1
--------------
Machine: 1 - 7
Machine: 4 - 3
--------------
--------------
Machine: 0 - 9
Machine: 1

256